In [1]:
import mlflow
# Connect to Databricks workspace with submitted credentials or use stored credentials
mlflow.login()
# Set tracking URI to Databricks -  tell MLflow to send the data into Databricks Workspace
mlflow.set_tracking_uri("databricks")

2025/10/30 01:31:15 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-7d1169bb-4536.cloud.databricks.com.


In [3]:
import os;
import mlflow
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

experiment_name = f"/Users/{os.environ['USER_EMAIL']}/agen-no-memory-{datetime.now()}"
try:
    mlflow.create_experiment(experiment_name)
except Exception as e:
    print(f"Experiment {experiment_name} already exists.")
mlflow.set_experiment(experiment_name)


with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        python_model="agent.py",
        name="Helpful agent without memory",
        registered_model_name="workspace.default.helpful-agent-no-memory",
    )

INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token


INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
2025/10/30 01:38:48 INFO mlflow.pyfunc: Predicting on input example to validate output
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token
INFO:code_model_6232461bf3b34a1284eb1fdfff6a093e:Last message type: message
INFO:code_model_6232461bf3b34a1284eb1fdfff6a093e:Calling LLM with messages: [{'role': 'system', 'content': 'You are a helpful assistant that can call tools to get information.'}, {'status': None, 'content'

🏃 View run loud-bear-445 at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/1420225184561539/runs/145d0be5f70c45be823ca922339ff946
🧪 View experiment at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/1420225184561539
